In [1]:
import numpy as np
import pandas as pd
import requests
import csv
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
pages = np.arange(1,1000,50)
#genres = ['sci-fi', 'comedy', 'action', 'thriller', 'drama', 'romance', 'crime', 'animation', 'sport', 'talk-show']

In [3]:
def get_url(search_movie):
    template = "https://www.imdb.com/search/title/?genres={}"
    url = template.format(search_movie)
    url += "&start={}"
    return url

def extract_details(movie):
    """Extract the movie details"""
    #Movie title
    title = movie.h3.a.text
    #MovieId
    movieID = movie.h3.a['href'][9:17]
    #Overview
    summary = movie.find_all('p', attrs={'class': "text-muted"})
    overview = summary[1].text
    #Year of release
    year = movie.h3.find('span', class_="lister-item-year text-muted unbold").text[1:5]
    #Movie duration   
    try:    
        duration = movie.p.find('span', attrs={'class': "runtime"}).text
    except AttributeError:
        duration = ''    
    #Genres
    try:
        genre = movie.p.find('span', attrs={'class': "genre"}).text
    except AttributeError:
        genre = ''
    #Certificates
    try:
        cert = movie.p.find('span', attrs={'class': "certificate"}).text
    except AttributeError:
        cert = ''    
    #Ratings
    try:    
        rating = movie.find('div', class_= "inline-block ratings-imdb-rating").text.replace('\n', '')
    except AttributeError:
        rating = ''
    #Votes
    try:
        votes = movie.find('span', attrs={'name': 'nv'}).text
    except AttributeError:
        votes = ''
    #Extract the urls of all images
    images = movie.img['src']
    result = (title, movieID, overview, year, duration, genre, rating, votes, cert, images)
    return result

def main(search_movie):
    path = "C:/Users/AL RHO/Desktop/Old/DS_Practice/chromedriver.exe"
    driver = webdriver.Chrome(path)
    
    records = []
    url = get_url(search_movie)
    
    for page in pages:
        driver.get(url.format(page))
        page = requests.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        movie_data = soup.find_all('div', attrs={'class': 'lister-item mode-advanced'})
        
        for item in movie_data:
            record = extract_details(item)
            if record:
                records.append(record)
    #Saving the data to a csv file
    with open('crime.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title','MovieID', 'Overview', 'YearofRelease', 'Runtime', 'Genre', 'Rating', 'Votes', 'Certificate', 'Images'])
        writer.writerows(records)

In [ ]:
main('crime')

C:\Users\AL RHO\AppData\Local\Temp\ipykernel_14724\3165887485.py:50: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
